# Project Elissa - NLP Playground


In [3]:
import re
import pandas as pd
from langdetect import detect, LangDetectException
import nltk
import spacy
import gensim
from gensim import corpora


In [5]:
df_products = pd.read_csv("../data/products.csv")
df_reviews = pd.read_csv("../data/reviews.csv")


## Language Detection


In [3]:
def detect_language(body: str | None) -> str:
    """Detect language of a string."""

    if body is None:
        return None
    try:
        return detect(body)
    except LangDetectException:
        return None


df_reviews["language"] = df_reviews["body"].apply(detect_language)


In [4]:
# Display languages that have more than 100 reviews
df_reviews["language"].value_counts().loc[lambda x: x > 100]


language
fr    3998
en    3575
de    2127
it    1325
es    1182
Name: count, dtype: int64

## Theme Modelling

- [Difference bewteen LSA and LDA - Medium](https://medium.com/@sujathamudadla1213/difference-between-lda-and-lsa-f7fefa6b4bfd)

We will proceed with only French for the time being.


In [5]:
LANG_EN = {"long": "english", "short": "en", "spacy": "en_core_web_sm"}
LANG = {"long": "french", "short": "fr", "spacy": "fr_core_news_sm"}


### Text Cleaning


In [16]:
nltk.download("stopwords")

df = df_reviews.query("language == 'en'")
stop_words = nltk.corpus.stopwords.words(LANG["long"])


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yunan.wang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
def clean_text(text: str | None) -> str:
    """
    Clean text by removing non-alphabetic characters,
    shortwords and multiple spaces.
    """

    if text is None:
        return None

    # remove everything except alphabets (accents included)
    text = re.sub(r"[^a-zA-ZÀ-ÿ]", " ", text)
    # remove shortwords (length <= 3)
    text = " ".join([word for word in text.split() if len(word) > 3])
    # remove multiple spaces
    text = " ".join(text.split())
    # lowercase
    text = text.lower()

    return text


In [18]:
def remove_stopwords(text: str | None) -> str:
    """Remove stopwords from text."""

    if text is None:
        return None

    text = " ".join([word for word in text.split() if word not in stop_words])

    return text


In [19]:
doc_cleaned = df["body"].apply(clean_text).apply(remove_stopwords)


### LSA (Latent Semantic Analysis)


In [20]:
def train_LSA(texts: pd.Series, lang: str = LANG["long"]) -> dict[int, list[str]]:
    """Train Latent Semantic Analysis model to extract topics from texts."""

    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.decomposition import TruncatedSVD

    stopwords = nltk.corpus.stopwords.words(lang)
    vectorizer = TfidfVectorizer(
        stop_words=stopwords,
        max_features=1000,
        max_df=0.5,
        smooth_idf=True,
    )

    X = vectorizer.fit_transform(texts)
    svd_model = TruncatedSVD(
        n_components=10, algorithm="randomized", n_iter=100, random_state=122
    )
    svd_model.fit(X)
    terms = vectorizer.get_feature_names_out()

    topics = {}
    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key=lambda x: x[1], reverse=True)[:7]
        topics[i] = [t[0] for t in sorted_terms]
    return topics


In [21]:
LSA_result = train_LSA(doc_cleaned)


In [22]:
LSA_result


{0: ['product', 'good', 'great', 'like', 'comfortable', 'easy', 'cups'],
 1: ['good', 'product', 'great', 'quality', 'price', 'value', 'comfortable'],
 2: ['great', 'product', 'easy', 'comfortable', 'love', 'organic', 'tampons'],
 3: ['comfortable',
  'easy',
  'product',
  'clean',
  'menstrual',
  'remove',
  'effective'],
 4: ['product', 'disc', 'first', 'using', 'time', 'period', 'inside'],
 5: ['love',
  'organic',
  'tampons',
  'applicator',
  'plastic',
  'comfortable',
  'cotton'],
 6: ['love', 'easy', 'clean', 'quality', 'works', 'good', 'recommend'],
 7: ['love', 'size', 'well', 'comfortable', 'small', 'liners', 'soft'],
 8: ['easy', 'small', 'well', 'size', 'liners', 'plastic', 'would'],
 9: ['size', 'small', 'applicator', 'tampons', 'perfect', 'large', 'organic']}

### LDA (Latent Dirichlet Allocation)


In [23]:
!python -m spacy download {LANG["spacy"]}
nlp = spacy.load(LANG["spacy"], disable=["parser", "ner"])


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.1 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [24]:
def lemmatize_text(text: str, tags=["NOUN", "ADJ", "VERB"]) -> list[str]:
    """Lemmatize text."""

    doc = nlp(text)
    return [token.lemma_ for token in doc if token.pos_ in tags]


In [25]:
doc_lemmatized = doc_cleaned.apply(lemmatize_text)


In [35]:
def train_LDA(docs: pd.Series) -> dict[str, object]:
    docs_lemmatized = docs.apply(lemmatize_text)
    dictionary = corpora.Dictionary(docs_lemmatized)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in docs_lemmatized]
    LDA = gensim.models.ldamodel.LdaModel
    lda_model = LDA(
        corpus=doc_term_matrix,
        id2word=dictionary,
        num_topics=10,
        random_state=100,
        chunksize=1000,
        passes=50,
    )

    topics_df = pd.DataFrame(
        {
            f"topic_{i}": [token for token, _ in lda_model.show_topic(i, topn=10)]
            for i in range(lda_model.num_topics)
        }
    )
    output = {
        "model": lda_model,
        "dictionary": dictionary,
        "doc_term_matrix": doc_term_matrix,
        "topics_df": topics_df,
    }
    return output


In [36]:
LDA_result = train_LDA(doc_cleaned)
LDA_result["topics_df"]


,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,product,false,liner,product,absorbent,tampon,use,clean,comfy,period
1,receive,review,pad,great,comfortable,applicator,cup,wash,wing,use
2,fall,playercard,thin,good,natural,organic,disc,smell,sticky,tampon
3,pack,vsemetric,stay,love,protection,brand,size,brand,amount,heavy
4,return,clientid,panty,recommend,arrive,one,small,use,moon,time
5,order,player,bag,use,flimsy,find,feel,work,ultra,feel
6,purchase,event,place,comfortable,pantie,plastic,try,good,usual,day
7,open,true,underwear,easy,chemical,make,menstrual,price,thick,hour
8,irritate,videoasinlist,stick,quality,unscented,cotton,easy,love,hygienic,leak
9,look,isinternal,wrap,price,package,use,make,skin,variety,take


In [39]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(
    topic_model=LDA_result["model"],
    corpus=LDA_result["doc_term_matrix"],
    dictionary=LDA_result["dictionary"],
)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.221910  0.089536       1        1  42.366210
9     -0.202600  0.115153       2        1  23.366789
3     -0.160288 -0.015835       3        1  10.639134
5     -0.108230  0.001590       4        1   6.223975
1      0.297927  0.285474       5        1   4.359263
0      0.043695 -0.076998       6        1   3.780520
2      0.014883 -0.120945       7        1   3.615846
7     -0.019981 -0.054740       8        1   2.927269
8      0.191824 -0.124679       9        1   1.518507
4      0.164680 -0.098557      10        1   1.202485, topic_info=            Term         Freq        Total Category  logprob  loglift
157      product  1159.000000  1159.000000  Default  30.0000  30.0000
137  comfortable   768.000000   768.000000  Default  29.0000  29.0000
101       tampon   777.000000   777.000000  Default  28.0000  28.0000
48         false   340.000000   340.000000  Default  27.0000  27.0000
291        great   681.000000   681.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
145      monthly     6.196878    16.443333  Topic10  -5.1963   3.4449
588       health     6.661761    20.932342  Topic10  -5.1240   3.2759
626        throw     7.288215    49.145268  Topic10  -5.0341   2.5123
427      purpose     6.263365    35.773339  Topic10  -5.1857   2.6783
574         item     6.228796    78.891179  Topic10  -5.1912   1.8819

[609 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
936      10  0.600590       abit
321       2  0.034212     absorb
321       4  0.530281     absorb
321       7  0.427646     absorb
136       9  0.940400  absorbant
...     ...       ...        ...
12        3  0.132186       year
12        4  0.014964       year
12        8  0.054870       year
2613      4  0.890382      yeast
1235      1  0.983555    youtube

[999 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 10, 4, 6, 2, 1, 3, 8, 9, 5])